In [1]:
import pandas as pd
import pickle as pkl
import re
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

import stanza
# stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

def evaluation_metrics(pred, gt):
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score 

def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

def get_term_(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for token, label in zip(tokens, labels):
            if label == 'B':
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # Check b_pos = 0 không
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms  

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hanhtran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2021-06-09 15:35:03 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-06-09 15:35:03 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-06-09 15:35:03 INFO: Use device: cpu
2021-06-09 15:35:03 INFO: Loading: tokenize
2021-06-09 15:35:03 INFO: Loading: pos
2021-06-09 15:35:03 INFO: Loading: lemma
2021-06-09 15:35:03 INFO: Done loading processors!


In [2]:
def get_term_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'
        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                    b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                    c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                    if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b_word.upos == 'ADJ'):
                        terms.append(' '.join([b_word.text] + term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

In [3]:
def get_term_noun_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0:
                    if (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b_word.upos == 'NOUN'):
                            terms.append(' '.join([b_word.text] + term))
                    if (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0):
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        if (a_word.text != 'None') and (a_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

In [4]:
def get_term_adj_noun_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                # Check b_pos = 0 không
                if b_pos != 0:
                    #ADJ NOUN NOUN
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[i])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        # Check vị trí b_pos - 1: terms.append()
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (b_word.text != 'None') and ((b_word.upos == 'ADJ') and (a_word.upos == 'NOUN')):
                            terms.append(' '.join([b_word.text] + term + [a_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # check b_pos - 1
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

In [5]:
def get_term_adj_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0:
                    #ADJ ADJ NOUN              
                    if (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'ADJ')):
                            terms.append(' '.join([b1_word.text] +[b_word.text] + term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

In [6]:
def get_term_noun_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0:
                    #NOUN ADJ NOUN
                    if (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'NOUN')):
                            terms.append(' '.join([b1_word.text] +[b_word.text] + term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

In [7]:
def get_term_noun_noun_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                # NOUN NOUN NOUN
                if b_pos != 0 and i + 1 < len(tokens):
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0 and len(nlp(str(tokens[i+1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0] 
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (a_word.text != 'None') and (a1_word.text != 'None') and ((a_word.upos == 'NOUN') and (a1_word.upos == 'NOUN')):
                            terms.append(' '.join( term + [a_word.text] +[a1_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

In [8]:
def get_term_noun_adp_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i+1 < len(tokens):
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0 and len(nlp(str(tokens[i+1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0] 
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN')and (c_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and ((a_word.upos == 'ADP') and (a1_word.upos == 'NOUN')):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text]))                               
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms

In [9]:
def get_term_noun_adp_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())
        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'
        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 2 < len(tokens):
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0 and len(nlp(str(tokens[i+1])).sentences) > 0 and len(nlp(str(tokens[i+2])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0] 
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0] 
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and (a2_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.upos == 'ADJ') and (a2_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text] +[a2_word.text]))   
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms    

In [10]:
def get_term_adj_noun_adp_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 1 < len(tokens):
                    if (tokens[i+1] != '') and (tokens[i+1] != ' ') and (tokens[i] != '') and (tokens[i] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and (len(nlp(str(tokens[i])).sentences) > 0) and (len(nlp(str(tokens[i+1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (b_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and (b_word.upos == 'ADJ') and (a_word.upos == 'ADP') and (a1_word.upos == 'NOUN'):
                            terms.append(' '.join([b_word.text] + term + [a_word.text] + [a1_word.text]))        
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms  

In [11]:
def get_term_noun_adp_noun_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 2 < len(tokens):
                    if (tokens[i+2] != '') and (tokens[i+2] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (len(nlp(str(tokens[i+1])).sentences) > 0) and (len(nlp(str(tokens[i+2])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                            a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                            a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                            a_word = nlp(str(tokens[i])).sentences[0].words[0]
                            c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                            if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (a_word.text != 'None')and (a1_word.text != 'None') and (a2_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.upos == 'NOUN') and (a2_word.upos == 'NOUN'):
                                terms.append(' '.join(term + [a_word.text] + [a1_word.text] + [a2_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # check b_pos - 1
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms  

In [12]:
def get_term_adj_noun_adp_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (i + 2 < len(tokens)) and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (tokens[i] != '') and (tokens[i] != ' ')and (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[i])).sentences) > 0) and (len(nlp(str(tokens[i+1])).sentences) > 0) and (len(nlp(str(tokens[i+2])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                    a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                    a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                    a_word = nlp(str(tokens[i])).sentences[0].words[0]
                    b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                    c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                    if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (b_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and (a2_word.text != 'None') and (b_word.upos == 'ADJ') and (a_word.upos == 'ADP') and (a1_word.upos == 'ADJ')and (a2_word.upos == 'NOUN'):
                        terms.append(' '.join([b_word.text] + term + [a_word.text] + [a1_word.text]  + [a2_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms  

In [13]:
def get_term_noun_pron(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (tokens[i] != '') and (tokens[i] != ' ') and (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        # Check vị trí b_pos - 1: terms.append()
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN')  and (a_word.text != 'None') and (a_word.upos == 'PROPN'):
                            terms.append(' '.join(term + [a_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # check b_pos - 1
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms  

In [14]:
def get_term_adv_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'
        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'ADV')):
                            terms.append(' '.join([b1_word.text] +[b_word.text] + term))                   
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms    

In [15]:
def get_term_adj_noun_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (i + 1 < len(tokens)) and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (len(nlp(str(tokens[i])).sentences) > 0) and  (len(nlp(str(tokens[i+1])).sentences) > 0): 
                    a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                    a_word = nlp(str(tokens[i])).sentences[0].words[0]
                    b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                    c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                    # Check vị trí b_pos - 1: terms.append()
                    if (c_word.text != 'None') and (b_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and (c_word.upos == 'NOUN') and ((b_word.upos == 'ADJ') and (a_word.upos == 'ADJ') and (a1_word.upos == 'NOUN')):
                        terms.append(' '.join([b_word.text] + term + [a_word.text] + [a1_word.text]))
                                
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # check b_pos - 1
        
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms

In [16]:
def term_evaluation(domain_path, preds_path, rule=None):
    groundtruth = pd.read_csv(domain_path, sep='	', engine='python',header=None)
    gt = list(groundtruth[0])
    predictions = pkl.load(open(preds_path, 'rb'))
    if rule == 'adj_noun':
        preds =  get_term_adj_noun(predictions)
    elif rule == 'noun_noun':
        preds =  get_term_noun_noun(predictions)
    elif rule == 'adj_adj_noun':
        preds = get_term_adj_adj_noun(predictions)
    elif rule == 'adj_noun_noun':
        preds = get_term_adj_noun_noun(predictions)
    elif rule == 'noun_adj_noun':
        preds = get_term_noun_adj_noun(predictions)
    elif rule == 'noun_noun_noun':
        preds = get_term_noun_noun_noun(predictions)
    elif rule == 'noun_adp_noun':
        preds = get_term_noun_adp_noun(predictions)
    elif rule == 'noun_adp_adj_noun':
        preds = get_term_noun_adp_adj_noun(predictions)
    elif rule == 'adj_noun_adp_noun':
        preds = get_term_adj_noun_adp_noun(predictions)
    elif rule == 'noun_adp_noun_noun':
        preds = get_term_noun_adp_noun_noun(predictions)
    elif rule == 'adj_noun_adp_adj_noun':
        preds = get_term_adj_noun_adp_adj_noun(predictions)
    elif rule == 'adv_adj_noun':
        preds = get_term_adv_adj_noun(predictions)
    elif rule == 'noun_pron':
        preds = get_term_noun_pron(predictions)
    elif rule == 'adj_noun_adj_noun':
        preds = get_term_adj_noun_adj_noun(predictions)
    else:
        preds =  get_term_(predictions)
    stop_words = set(stopwords.words('english'))
    pred_terms =  set(preds) - set(stop_words)
    pred_terms = [x for x in pred_terms if len(x)>1]
    pred_terms = [x.lower().strip() for x in pred_terms]
    pred_terms = [re.sub(' -','-', x) for x in pred_terms]
    pred_terms = [re.sub('- ','-', x) for x in pred_terms]
    pred_terms = [re.sub('\(','', x) for x in pred_terms]
    pred_terms = [re.sub('\/','', x) for x in pred_terms]
    precision, recall, f1 = evaluation_metrics(pred_terms, gt)
    return precision, recall, f1
    

In [17]:
path = '/Users/hanhtran/Documents/terminology-extraction/ACTER/'
domain_paths = [path+'en/htfl/annotations/htfl_en_terms.ann', path+'en/wind/annotations/wind_en_terms.ann',
                path+'en/equi/annotations/equi_en_terms.ann',path+'en/corp/annotations/corp_en_terms.ann',
                path+'en/htfl/annotations/htfl_en_terms_nes.ann', path+'en/wind/annotations/wind_en_terms_nes.ann',
                path+'en/equi/annotations/equi_en_terms_nes.ann',path+'en/corp/annotations/corp_en_terms_nes.ann',
               ]
preds_paths = ['ann_xlnet_htfl.pkl','ann_xlnet_wind.pkl','ann_xlnet_equi.pkl', 'ann_xlnet_corp.pkl',
              'nes_xlnet_htfl.pkl','nes_xlnet_wind.pkl','nes_xlnet_equi.pkl', 'nes_xlnet_corp.pkl']
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p)
    results.append([p,pre, rec, f1])
raw_res = pd.DataFrame(results,columns=['DOMAIN','NON_PATTERN_P','NON_PATTERN_R','NON_PATTERN_F1'])
raw_res

,DOMAIN,NON_PATTERN_P,NON_PATTERN_R,NON_PATTERN_F1
0,ann_xlnet_htfl.pkl,42.94,30.03,35.34
1,ann_xlnet_wind.pkl,37.76,52.61,43.96
2,ann_xlnet_equi.pkl,42.22,61.99,50.23
3,ann_xlnet_corp.pkl,42.54,59.33,49.55
4,nes_xlnet_htfl.pkl,43.63,36.83,39.94
5,nes_xlnet_wind.pkl,42.47,49.61,45.76
6,nes_xlnet_equi.pkl,46.77,55.17,50.62
7,nes_xlnet_corp.pkl,45.05,55.41,49.70


In [18]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'adj_noun')
    results.append([p,pre, rec, f1])
adj_noun = pd.DataFrame(results,columns=['DOMAIN','ADJ_NOUN_P','ADJ_NOUN_R','ADJ_NOUN_F1'])
adj_noun

KeyboardInterrupt: 

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1  = term_evaluation(d,p, 'noun_noun')
    results.append([p,pre, rec, f1])
noun_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN_P','NOUN_NOUN_R','NOUN_NOUN_F1'])
noun_noun

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'adj_noun_noun')
    results.append([p,pre, rec, f1])
adj_noun_noun = pd.DataFrame(results,columns=['DOMAIN','ADJ_NOUN_NOUN_P','ADJ_NOUN_NOUN_R','ADJ_NOUN_NOUN_F1'])
adj_noun_noun

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'adj_adj_noun')
    results.append([p,pre, rec, f1])
adj_adj_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADJ_NOUN_P','NOUN_ADJ_NOUN_R','NOUN_ADJ_NOUN_F1'])
adj_adj_noun

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'noun_adj_noun')
    results.append([p,pre, rec, f1])
noun_adj_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN_NOUN_P','NOUN_NOUN_NOUN_R','NOUN_NOUN_NOUN_F1'])
noun_adj_noun

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'noun_noun_noun')
    results.append([p,pre, rec, f1])
noun_noun_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN_NOUN_P','NOUN_NOUN_NOUN_R','NOUN_NOUN_NOUN_F1'])
noun_noun_noun

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'noun_adp_noun')
    results.append([p,pre, rec, f1])
noun_adp_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN_NOUN_P','NOUN_NOUN_NOUN_R','NOUN_NOUN_NOUN_F1'])
noun_adp_noun

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'noun_adp_adj_noun')
    results.append([p,pre, rec, f1])
noun_adp_adj_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN_NOUN_P','NOUN_NOUN_NOUN_R','NOUN_NOUN_NOUN_F1'])
noun_adp_adj_noun

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'noun_adp_noun_noun')
    results.append([p,pre, rec, f1])
noun_adp_noun_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN_NOUN_P','NOUN_NOUN_NOUN_R','NOUN_NOUN_NOUN_F1'])
noun_adp_noun_noun

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'adj_noun_adp_adj_noun')
    results.append([p,pre, rec, f1])
adj_noun_adp_adj_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN_NOUN_P','NOUN_NOUN_NOUN_R','NOUN_NOUN_NOUN_F1'])
adj_noun_adp_adj_noun

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'adv_adj_noun')
    results.append([p,pre, rec, f1])
adv_adj_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN_NOUN_P','NOUN_NOUN_NOUN_R','NOUN_NOUN_NOUN_F1'])
adv_adj_noun

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'noun_pron')
    results.append([p,pre, rec, f1])
noun_pron = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN_NOUN_P','NOUN_NOUN_NOUN_R','NOUN_NOUN_NOUN_F1'])
noun_pron

In [ ]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p, 'adj_noun_adj_noun')
    results.append([p,pre, rec, f1])
adj_noun_adj_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN_NOUN_P','NOUN_NOUN_NOUN_R','NOUN_NOUN_NOUN_F1'])
adj_noun_adj_noun

In [ ]:
df_ls = [adj_noun, noun_noun, adj_adj_noun, adj_noun_noun, noun_adj_noun, 
        noun_noun_noun , noun_adp_noun, noun_adp_adj_noun, adj_noun_adp_noun, noun_adp_noun_noun,
        adj_noun_adp_adj_noun, adv_adj_noun, noun_pron, adj_noun_adj_noun]

In [ ]:
for d in df_ls:
    raw_res = raw_res.merge(d, on='DOMAIN', how='left')
raw_res